In [49]:
# Load env variables and create client
import os
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()


client = Anthropic(
  api_key=os.getenv("ZHIPU_API_KEY"),
  base_url="https://open.bigmodel.cn/api/anthropic"
)

model = "glm-4.7"

In [50]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)

def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [51]:
from datetime import datetime
from anthropic.types import ToolParam
def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    if not date_format:
        raise ValueError("date_format cannot empty")
    return datetime.now().strftime(date_format)



get_current_datetime_schema = ToolParam({
    "name": "get_current_datetime",
    "description": "Returns the current date and time formatted according to the specified format",
    "input_schema": {
        "type": "object",
        "properties": {
            "date_format": {
                "type": "string",
                "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes.",
                "default": "%Y-%m-%d %H:%M:%S"
            }
        },
        "required": []
    }
})



In [52]:
messages = []
messages.append(
    {
        "role": "user",
        "content": "what is the exact time, formatted as HH:MM:SS?"
    }
)

response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[
        get_current_datetime_schema
    ]
)

messages.append({
    "role": "assistant",
    "content": response.content
})

messages

[{'role': 'user', 'content': 'what is the exact time, formatted as HH:MM:SS?'},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='call_f83ecc2768a04229acaedc21', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]}]

In [53]:
result = get_current_datetime(**response.content[0].input)

In [54]:
messages.append({
    "role": "user",
    "content": [
        {
            "type": "tool_result",
            "tool_use_id": response.content[0].id,
            "content": result,
            "is_error": False
        }
    ]
})

messages

[{'role': 'user', 'content': 'what is the exact time, formatted as HH:MM:SS?'},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='call_f83ecc2768a04229acaedc21', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'call_f83ecc2768a04229acaedc21',
    'content': '00:06:11',
    'is_error': False}]}]

In [57]:
client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[
        get_current_datetime_schema
    ]
)

Message(id='msg_20260126220958eb6c84bff55d4c78', content=[TextBlock(citations=None, text='The exact time is **00:06:11**.', type='text')], model='glm-4.7', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=None, cache_creation_input_tokens=None, cache_read_input_tokens=0, input_tokens=236, output_tokens=13, server_tool_use=ServerToolUsage(web_search_requests=0), service_tier='standard'))